In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [2]:
train_df = pd.read_csv('train_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [3]:
test_df = pd.read_csv('test_tweets.csv')
test_df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [4]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [5]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


1. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()


In [6]:
def remove_user(data: pd.DataFrame):
    data['tweet'] = list(map(lambda x: re.sub(r"@[\w]*", " ", x), data['tweet'].values))

In [7]:
remove_user(combine_df)
combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so se...
1,2,0.0,thanks for #lyft credit i can't use cause ...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


2. Изменим регистр твитов на нижний с помощью .lower()

In [8]:
combine_df['tweet'] = list(map(str.lower, combine_df['tweet'].values))
combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so se...
1,2,0.0,thanks for #lyft credit i can't use cause ...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


3. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [9]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [10]:
def apostrophe_replace(text):
    res = ""
    for word in text.split():
        if word in apostrophe_dict.keys():
            res += apostrophe_dict[word]
        else:
            res += word
        res += " "
    return res[:-1]

In [11]:
apostrophe_replace(combine_df['tweet'].values[1])

'thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked'

In [12]:
combine_df['tweet'] = list(map(apostrophe_replace, combine_df['tweet'].values))
combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for #lyft credit i cannot use cause the...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ur...
4,5,0.0,factsguide: society now #motivation


4. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [13]:
short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

In [14]:
def shortword_replace(text):
    res = ""
    for word in text.split():
        if word in apostrophe_dict.keys():
            res += apostrophe_dict[word]
        else:
            res += word
        res += " "
    return res[:-1]

In [15]:
combine_df['tweet'] = list(map(shortword_replace, combine_df['tweet'].values))
combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for #lyft credit i cannot use cause the...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ur...
4,5,0.0,factsguide: society now #motivation


5. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [16]:
emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [17]:
def emogi_replace(text):
    res = ""
    for word in text.split():
        if word in apostrophe_dict.keys():
            res += apostrophe_dict[word]
        else:
            res += word
        res += " "
    return res[:-1]

In [18]:
combine_df['tweet'] = list(map(emogi_replace, combine_df['tweet'].values))
combine_df.tail()

,id,label,tweet
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...
49157,49158,NaN,"happy, at work conference: right mindset leads..."
49158,49159,NaN,"my song ""so glad"" free download! #shoegaze #ne..."


6. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

In [19]:
combine_df["tweet"] = list(map(lambda x: re.sub(r'[^\w\s]', "", x), combine_df["tweet"].values))
combine_df.head(10)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for lyft credit i cannot use cause they...
2,3,0.0,bihday your majesty
3,4,0.0,model i love u take with u all the time in urð...
4,5,0.0,factsguide society now motivation
5,6,0.0,22 huge fan fare and big talking before they l...
6,7,0.0,camping tomorrow dannyâ
7,8,0.0,the next school year is the year for examsð ca...
8,9,0.0,we won love the land allin cavs champions clev...
9,10,0.0,welcome here I am it has it is so gr8


7. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [20]:
combine_df["tweet"] = list(map(lambda x: re.sub(r'[^a-zA-Z0-9]', " ", x), combine_df["tweet"].values))
combine_df.tail(10)

,id,label,tweet
49149,49150,NaN,loving life createyourfuture lifestyle h...
49150,49151,NaN,black professor demonizes proposes nazi style ...
49151,49152,NaN,learn how to think positive positive instagram...
49152,49153,NaN,we love the pretty happy and fresh you teenili...
49153,49154,NaN,2 damn tuffruff muff techno cityng005web1997u...
49154,49155,NaN,thought factory leftright polarisation trump u...
49155,49156,NaN,feeling like a mermaid hairflip neverready f...
49156,49157,NaN,hillary campaigned today in ohioomg amp used w...
49157,49158,NaN,happy at work conference right mindset leads t...
49158,49159,NaN,my song so glad free download shoegaze newmusi...


8. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

In [21]:
combine_df["tweet"] = list(map(lambda x: re.sub(r'[^a-zA-Z]', " ", x), combine_df["tweet"].values))
combine_df.tail(10)

,id,label,tweet
49149,49150,NaN,loving life createyourfuture lifestyle h...
49150,49151,NaN,black professor demonizes proposes nazi style ...
49151,49152,NaN,learn how to think positive positive instagram...
49152,49153,NaN,we love the pretty happy and fresh you teenili...
49153,49154,NaN,damn tuffruff muff techno cityng web u...
49154,49155,NaN,thought factory leftright polarisation trump u...
49155,49156,NaN,feeling like a mermaid hairflip neverready f...
49156,49157,NaN,hillary campaigned today in ohioomg amp used w...
49157,49158,NaN,happy at work conference right mindset leads t...
49158,49159,NaN,my song so glad free download shoegaze newmusi...


9. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [22]:
combine_df["tweet"] = list(map(lambda x: ' '.join([w for w in x.split() if len(w)>1]), combine_df["tweet"].values))
combine_df.tail(10)

,id,label,tweet
49149,49150,NaN,loving life createyourfuture lifestyle holiday...
49150,49151,NaN,black professor demonizes proposes nazi style ...
49151,49152,NaN,learn how to think positive positive instagram...
49152,49153,NaN,we love the pretty happy and fresh you teenili...
49153,49154,NaN,damn tuffruff muff techno cityng web ukhx int ...
49154,49155,NaN,thought factory leftright polarisation trump u...
49155,49156,NaN,feeling like mermaid hairflip neverready forma...
49156,49157,NaN,hillary campaigned today in ohioomg amp used w...
49157,49158,NaN,happy at work conference right mindset leads t...
49158,49159,NaN,my song so glad free download shoegaze newmusi...


10. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [23]:
from nltk.tokenize import word_tokenize

In [24]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
combine_df['tweet_token'] = list(map(word_tokenize, combine_df["tweet"].values))
combine_df.tail(10)

,id,label,tweet,tweet_token
49149,49150,NaN,loving life createyourfuture lifestyle holiday...,"[loving, life, createyourfuture, lifestyle, ho..."
49150,49151,NaN,black professor demonizes proposes nazi style ...,"[black, professor, demonizes, proposes, nazi, ..."
49151,49152,NaN,learn how to think positive positive instagram...,"[learn, how, to, think, positive, positive, in..."
49152,49153,NaN,we love the pretty happy and fresh you teenili...,"[we, love, the, pretty, happy, and, fresh, you..."
49153,49154,NaN,damn tuffruff muff techno cityng web ukhx int ...,"[damn, tuffruff, muff, techno, cityng, web, uk..."
49154,49155,NaN,thought factory leftright polarisation trump u...,"[thought, factory, leftright, polarisation, tr..."
49155,49156,NaN,feeling like mermaid hairflip neverready forma...,"[feeling, like, mermaid, hairflip, neverready,..."
49156,49157,NaN,hillary campaigned today in ohioomg amp used w...,"[hillary, campaigned, today, in, ohioomg, amp,..."
49157,49158,NaN,happy at work conference right mindset leads t...,"[happy, at, work, conference, right, mindset, ..."
49158,49159,NaN,my song so glad free download shoegaze newmusi...,"[my, song, so, glad, free, download, shoegaze,..."


11. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [26]:
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [27]:
combine_df['tweet_token_filtered'] = list(map(lambda x: [word for word in x if not word in stopwords.words("english")], 
                                        combine_df["tweet_token"].values))

In [28]:
combine_df.head(10)

,id,label,tweet,tweet_token,tweet_token_filtered
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]"
3,4,0.0,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]"
5,6,0.0,huge fan fare and big talking before they leav...,"[huge, fan, fare, and, big, talking, before, t...","[huge, fan, fare, big, talking, leave, chaos, ..."
6,7,0.0,camping tomorrow danny,"[camping, tomorrow, danny]","[camping, tomorrow, danny]"
7,8,0.0,the next school year is the year for exams can...,"[the, next, school, year, is, the, year, for, ...","[next, school, year, year, exams, think, schoo..."
8,9,0.0,we won love the land allin cavs champions clev...,"[we, won, love, the, land, allin, cavs, champi...","[love, land, allin, cavs, champions, cleveland..."
9,10,0.0,welcome here am it has it is so gr,"[welcome, here, am, it, has, it, is, so, gr]","[welcome, gr]"


12. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [29]:
from nltk.stem import PorterStemmer

In [30]:
stemmer = PorterStemmer()

In [31]:
[stemmer.stem(word) for word in combine_df["tweet_token_filtered"].values[0]]

['father', 'dysfunct', 'selfish', 'drag', 'kid', 'dysfunct', 'run']

In [32]:
combine_df['tweet_stemmed'] = list(map(lambda x: set(stemmer.stem(word) for word in x), combine_df["tweet_token_filtered"].values))

In [33]:
combine_df.head(10)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","{selfish, father, kid, dysfunct, drag, run}"
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","{van, getthank, offer, lyft, credit, pdx, whee..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","{majesti, bihday}"
3,4,0.0,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]","{model, time, ur, love, take}"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","{factsguid, motiv, societi}"
5,6,0.0,huge fan fare and big talking before they leav...,"[huge, fan, fare, and, big, talking, before, t...","[huge, fan, fare, big, talking, leave, chaos, ...","{talk, big, huge, disput, fan, get, pay, fare,..."
6,7,0.0,camping tomorrow danny,"[camping, tomorrow, danny]","[camping, tomorrow, danny]","{tomorrow, danni, camp}"
7,8,0.0,the next school year is the year for exams can...,"[the, next, school, year, is, the, year, for, ...","[next, school, year, year, exams, think, schoo...","{imagin, hate, next, girl, school, revolutions..."
8,9,0.0,we won love the land allin cavs champions clev...,"[we, won, love, the, land, allin, cavs, champi...","[love, land, allin, cavs, champions, cleveland...","{clevelandcavali, cleveland, land, love, champ..."
9,10,0.0,welcome here am it has it is so gr,"[welcome, here, am, it, has, it, is, so, gr]","[welcome, gr]","{welcom, gr}"


13. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [34]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [36]:
lemmatizer = WordNetLemmatizer()

In [37]:
combine_df['tweet_lemmatized'] = list(map(lambda x: set(lemmatizer.lemmatize(word, pos=wordnet.VERB) for word in x), 
                                    combine_df["tweet_token_filtered"].values))

In [38]:
combine_df.head(10)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","{selfish, father, kid, dysfunct, drag, run}","{selfish, father, dysfunctional, kid, dysfunct..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","{van, getthank, offer, lyft, credit, pdx, whee...","{offer, cause, lyft, credit, disapointed, vans..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","{majesti, bihday}","{majesty, bihday}"
3,4,0.0,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]","{model, time, ur, love, take}","{model, time, ur, love, take}"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","{factsguid, motiv, societi}","{motivation, factsguide, society}"
5,6,0.0,huge fan fare and big talking before they leav...,"[huge, fan, fare, and, big, talking, before, t...","[huge, fan, fare, big, talking, leave, chaos, ...","{talk, big, huge, disput, fan, get, pay, fare,...","{talk, big, huge, fan, chaos, dispute, get, pa..."
6,7,0.0,camping tomorrow danny,"[camping, tomorrow, danny]","[camping, tomorrow, danny]","{tomorrow, danni, camp}","{danny, tomorrow, camp}"
7,8,0.0,the next school year is the year for exams can...,"[the, next, school, year, is, the, year, for, ...","[next, school, year, year, exams, think, schoo...","{imagin, hate, next, girl, school, revolutions...","{actorslife, hate, next, girl, school, revolut..."
8,9,0.0,we won love the land allin cavs champions clev...,"[we, won, love, the, land, allin, cavs, champi...","[love, land, allin, cavs, champions, cleveland...","{clevelandcavali, cleveland, land, love, champ...","{cavs, clevelandcavaliers, cleveland, land, lo..."
9,10,0.0,welcome here am it has it is so gr,"[welcome, here, am, it, has, it, is, so, gr]","[welcome, gr]","{welcom, gr}","{welcome, gr}"


14. Сохраним результат предобработки в pickle-файл.

In [39]:
import pickle

In [40]:
combine_df.loc[combine_df['label'] != -1].to_pickle("train.pkl")

In [41]:
combine_df.loc[combine_df['label'] == -1].drop('label', axis=1).head().to_pickle("test.pkl")